In [ ]:
from impresso import connect

impresso = connect()

# Named entity recognition


In [2]:
text = """
Jean-Baptiste Nicolas Robert Schuman ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = impresso.tools.ner(
    text=text
)
result

,type,surfaceForm,function,name,confidence.ner,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName
id,,,,,,,,,
1:37:pers:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,pers,Jean-Baptiste Nicolas Robert Schuman,N/A,Baptiste Nicolas Robert Schuman,93.81,1,37,N/A,N/A
41:53:time:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,time,29 June 1886,N/A,N/A,86.49,41,53,N/A,N/A
56:72:time:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,time,4 September 1963,N/A,N/A,74.53,56,72,N/A,N/A


In [3]:
text = """
Jean-Baptiste Nicolas Robert Schuman ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = impresso.tools.ner_nel(
    text=text,
)
result

,type,surfaceForm,function,name,confidence.ner,confidence.nel,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName,wikidata.wikipediaPageUrl
id,,,,,,,,,,,
1:37:pers:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,pers,Jean-Baptiste Nicolas Robert Schuman,N/A,Baptiste Nicolas Robert Schuman,93.81,99.57,1,37,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
41:53:time:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,time,29 June 1886,N/A,N/A,86.49,93.57,41,53,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
56:72:time:ner-stacked-2-bert-medium-historic-multilingual|ner-mgenre-multilingual,time,4 September 1963,N/A,N/A,74.53,81.87,56,72,NaN,NaN,NaN


In [4]:
text = """
[START]Jean-Baptiste Nicolas Robert Schuman[END] ( 
29 June 1886 – 4 September 1963) was a Luxembourg-born French 
statesman. Schuman was a Christian democratic (Popular 
Republican Movement) political thinker and activist. 
"""
result = impresso.tools.nel(
    text=text,
)
result

,type,surfaceForm,confidence.nel,offset.start,offset.end,wikidata.id,wikidata.wikipediaPageName,wikidata.wikipediaPageUrl
id,,,,,,,,
8:44:UNK:nel-mgenre-multilingual,unk,Jean-Baptiste Nicolas Robert Schuman,99.94,8,44,Q15981,Robert Schuman,https://en.wikipedia.org/wiki/Robert_Schuman
